In [35]:
from examwiz_pkg.examwiz_pkg.gapi_app import grade_book as gb

#### 1. Exam is Created
Admin writes new exam (or is porting current exam into the system)

In [40]:
#exam2 = gb.create_grade_book('Python Midterm2', in_domain=True)
## Generates new exam that is view only to SupStat Inc users, but can be edited by owner.
exam = '1ZuM7K8ygsAhBjlBr2FE3zyilRwJpT34XLId602OuETE'
print('New Gradebook created. Access at:', gb.get_link(exam))

#Admin links gradebook to a google form and manually structures the questions (needs to be automated)
form_link = 'https://docs.google.com/forms/d/e/1FAIpQLSex4HTuoBouCkkN0XrszdDiqzGVOZwocaTdQKd0zhNxot5SHA/viewform?usp=sf_link'

New Gradebook created. Access at: https://docs.google.com/spreadsheets/d/1ZuM7K8ygsAhBjlBr2FE3zyilRwJpT34XLId602OuETE/edit?usp=drivesdk


#### 2. Exam is administered
Cohort students take the exam and provide their solutions to the portal. This is downloadeded from the portal and run through an anonymizer

In [34]:
from examwiz_pkg.examwiz_pkg.anon_app import anonymizer as an
gb.fresh_start()
raw_path = 'demo/demo files/raw_student_exams'
print('Original Student Submissions:', an.list_files(raw_path))
anon_path, key = an.make_anon(raw_path)
print('\nAnonymized Submissions:', an.list_files(anon_path))
print('\nConversion Key:\n', an.read_key(raw_path+'/../conversion_key.csv').head(2))

Original Student Submissions: ['MarioValdez_exam.R', 'MikeGriffin_Exam.R', 'Luana Stamato Caldeira.R', 'WillPonsby_RExam.R', 'PaulLee.R', 'TedDuncayExam.R', 'Bee_Kim_RExam2.R', 'Brian_Zhou_RExam.R', 'Aron_Berke_Exam_Code.R', 'Austin_Cheng_Exam.R', 'MikolajWilkExam.R', 'DrucilaLebfrevreExam.R', 'IraVillarMidTerm.R', 'Catherine_E_RExam.R', 'TomasNevinsExam.R']

Anonymized Submissions: ['7722.R', '9218.R', '6362.R', '5523.R', '8390.R', '5355.R', '3277.R', '1307.R', '6400.R', '9599.R', '2713.R', '6799.R', '8088.R', '2244.R', '8107.R']

Conversion Key:
    Unnamed: 0                                                new  \
0           0  demo/demo files/raw_student_exams/../anon_stud...   
1           1  demo/demo files/raw_student_exams/../anon_stud...   

                                                orig  temp_id  \
0  demo/demo files/raw_student_exams/MarioValdez_...     8088   
1  demo/demo files/raw_student_exams/MikeGriffin_...     8107   

                         time  
0  2020-01-0

#### 3. Send Anonymized Exams to TAs
Randomly split the examinations and send the files to the TAs along with the grading form.

In [31]:
from examwiz_pkg.examwiz_pkg.gapi_app import mailer as ml

#Find Active TAs
names, emails = ml.get_ta_emails('demo/demo files/TA_data.txt')
num_tas = len(names)
num_exams = len(an.list_files(anon_path))

#Email TAs student files and grading form link
for i in range(num_tas):
    exams = an.list_files(anon_path)[(i*num_exams//num_tas):((i+1)*num_exams//num_tas)]
    em = ml.create_attached_message(
        sender='charles.cohen@nycdatascience.com',
        to=emails[i],
        subject=f'{names[i]}\'s to grade: R Midterm',
        msg=f"""Hi {names[i]},
Here are the R Midterm student exams.
Please submit your grade to them promptly.
Here is the submission form link {form_link}""",
        file_dir = 'demo/demo files/anon_student_exams',
        filenames = exams
    )
    
    ml.send_message(em)

Message Sent!
Message Sent!
Message Sent!
Message Sent!


#### 4. Admin audits gradebook (manually)

#### 5. Admin flags gradebook for reporting (manually)

#### 6. Generates exam reports and emails to students. (automatic)